## ETL

In [ ]:
def extract_titanic_data(url, refresh_cache=False):
    cache_fn = Path('/dataset/titanic_local.csv')
    if not cache_fn.exists() or refresh_cache:
        print("Getting data")
        df = pd.read_csv(url)
        df.to_csv(cache_fn, index=False)
    print("Using cache")
    df = spark.read.csv(str(cache_fn), header=True)
    return df

In [ ]:
url = 'https://gist.githubusercontent.com/petobens/29825af615da09752c2817e96732215a/raw/5ad193b26e8a2b79220028f0f1520a2d868ab28e/titanic.csv'